In [5]:
import os
import tensorflow as tf
import cv2

In [6]:
print(tf.__version__)

2.9.1


In [7]:
import tensorflow.keras.preprocessing.image as Image

In [8]:
categories=['with_mask','without_mask']

In [9]:
data=[]
for category in categories:
    label=categories.index(category)
    path=os.path.join('train',category)
    for file in os.listdir(path):
        img_path=os.path.join(path,file)
        img=cv2.imread(img_path)
        img=cv2.resize(img,(224,224))
        data.append([img,label])
        
        

In [10]:
len(data)

1279

In [11]:
import random
random.shuffle(data)

In [12]:
X=[]
y=[]
for features,labels in data:
    X.append(features)
    y.append(labels)

In [13]:
len(X)

1279

In [14]:
len(y)

1279

In [15]:
import numpy as np

In [16]:
X=np.array(X)
y=np.array(y)

In [17]:
X.shape

(1279, 224, 224, 3)

In [18]:
y.shape

(1279,)

In [19]:
X=X/255

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [22]:
len(X_train)

1023

In [23]:
len(X_test)

256

In [24]:
from tensorflow.keras.applications import MobileNetV2

In [25]:
model=MobileNetV2(weights='imagenet',include_top=False)

In [26]:
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 Conv1 (Conv2D)                 (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, None, None,   128         ['Conv1[0][0]']                  
                                32)                                            

In [27]:
from tensorflow.keras import Sequential,Model
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D

In [28]:
x=model.output
x=GlobalAveragePooling2D()(x)
x=Dense(50,activation='relu')(x)
preds=Dense(1,activation='sigmoid')(x)

model_transfer=Model(inputs=model.input,outputs=preds)


In [29]:
model_transfer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 Conv1 (Conv2D)                 (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, None, None,   128         ['Conv1[0][0]']                  
                                32)                                                           

In [30]:
for layers in model_transfer.layers[:-5]:
    layers.trainable=False
    

In [31]:
model_transfer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 Conv1 (Conv2D)                 (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, None, None,   128         ['Conv1[0][0]']                  
                                32)                                                           

In [34]:
model_transfer.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [35]:
model_transfer.fit(X_train,y_train,epochs=3,validation_data=(X_test,y_test))

Epoch 1/3
32/32 [==============================] - 33s 926ms/step - loss: 0.1573 - accuracy: 0.9365 - val_loss: 0.0582 - val_accuracy: 0.9766
Epoch 2/3
32/32 [==============================] - 27s 848ms/step - loss: 0.0315 - accuracy: 0.9912 - val_loss: 0.0662 - val_accuracy: 0.9805
Epoch 3/3
32/32 [==============================] - 27s 859ms/step - loss: 0.0204 - accuracy: 0.9932 - val_loss: 0.0200 - val_accuracy: 0.9922


In [36]:
def detect_facemask(img):
    y_pred=model_transfer.predict(img.reshape(1,224,224,3))
    return np.round(y_pred)[0][0]


In [37]:
detect_facemask(X[0])

1/1 [==============================] - 1s 760ms/step


0.0

In [38]:
y[0]

0

In [39]:
sample=cv2.imread('/Users/abhishek/Desktop/PYTHON/Face Mask detection/OIP.jpeg')
sample=cv2.resize(sample,(224,224))
detect_facemask(sample)

1/1 [==============================] - 1s 678ms/step


0.0

In [40]:
sample=cv2.imread('685BE1AE-0BDB-4A78-832C-0A6D68D934E8_1_105_c.jpeg')
sample=cv2.resize(sample,(224,224))
detect_facemask(sample)

1/1 [==============================] - 0s 43ms/step


1.0

In [41]:
sample=cv2.imread('woman-wearing-mask.original.jpg')
sample=cv2.resize(sample,(224,224))
detect_facemask(sample)

1/1 [==============================] - 0s 38ms/step


1.0

In [42]:
def draw_label(img,text,pos,bg_color):

    text_size=cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,2,cv2.FILLED)
    end_x=pos[0]+text_size[0][0]+2
    end_y=pos[1]+text_size[0][0]+2
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),cv2.LINE_AA)


In [1]:
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    frame=cv2.resize(frame,(224,224))
    y_pred=detect_facemask(frame)
    if y_pred==0:
        draw_label(frame,"Mask",(30,30),(255,0,0))
    else:
        draw_label(frame,"No Mask",(30,30),(0,0,255))

    cv2.imshow('window',frame)

    if cv2.waitKey(1) & 0xFF==ord('w'):
        break
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined